In [1]:
BROKERS = ["host.docker.internal:9092"]

In [9]:
#from kafka import KafkaProducer, KafkaConsumer
import json
import threading
import time
import uuid
from pcomp_utils.kafka_confluent_utils import KafkaProducerHandler, KafkaConsumerHandler

KAFKA_BROKER = 'kafka:9092'

def produce_messages():
    # producer = KafkaProducer(
    #     bootstrap_servers=BROKERS,
    #     value_serializer=lambda v: json.dumps(v).encode('utf-8')
    # )
    producer = KafkaProducerHandler(KAFKA_BROKER)

    for i in range(5):
        message = {
            "user_id": i,
            "action": "activate",
            "timestamp": "2025-03-12T12:00:00"
        }
        producer.send('activate-layer', message, 0)
        print("Message sent: ", message)
        time.sleep(1)

    producer.close()

def consume_messages():
    consumer = KafkaConsumerHandler('activate-layer', KAFKA_BROKER, partition=0)

    print("Starting consumer...")
    count = 0
    for message in consumer.consume():
        time.sleep(2)
        layer= message.get("layer")
        print(f"Received message: {layer}")
        count += 1
        if count >= 5:
            break

if __name__ == "__main__":
    # Start consumer in a separate thread
    consumer_thread = threading.Thread(target=consume_messages)
    consumer_thread.start()

    # Ensure the consumer is fully ready
    time.sleep(2)

    produce_messages()

    consumer_thread.join()


Starting consumer...
Message sent:  {'user_id': 0, 'action': 'activate', 'timestamp': '2025-03-12T12:00:00'}
Received message: layer_0
Message sent:  {'user_id': 1, 'action': 'activate', 'timestamp': '2025-03-12T12:00:00'}
Message sent:  {'user_id': 2, 'action': 'activate', 'timestamp': '2025-03-12T12:00:00'}
Received message: layer_0
Message sent:  {'user_id': 3, 'action': 'activate', 'timestamp': '2025-03-12T12:00:00'}
Message sent:  {'user_id': 4, 'action': 'activate', 'timestamp': '2025-03-12T12:00:00'}
Received message: layer_0
Received message: layer_0
Received message: layer_0


In [ ]:
import time
from pcomp_utils.kafka_confluent_utils import KafkaProducerHandler, KafkaConsumerHandler

KAFKA_BROKER = 'kafka:9092'
TOPIC_NAME = 'activate-layer'
GROUP_ID = 'simple-group'

# Simple Kafka Producer
def simple_producer():
    producer = KafkaProducerHandler(KAFKA_BROKER)
    for i in range(10):
        message = {'key': f'message-{i}', 'value': f'This is message number {i}'}
        producer.send(TOPIC_NAME, message)
        print(f'Produced: {message}')
        time.sleep(1)
    producer.close()

# Simple Kafka Consumer
def simple_consumer():
    consumer = KafkaConsumerHandler(TOPIC_NAME, KAFKA_BROKER, group_id=GROUP_ID)
    print('Starting consumer...')
    try:
        for message in consumer.consume():
            print(f'Consumed: {message.value()}')
    except KeyboardInterrupt:
        print("Consumer interrupted")
    finally:
        consumer.close()

if __name__ == '__main__':
    from threading import Thread

    # Start the consumer in a separate thread
    consumer_thread = Thread(target=simple_consumer)
    consumer_thread.start()

    # Give the consumer a moment to start
    time.sleep(2)

    # Run the producer
    simple_producer()

    # Wait for the consumer to finish
    consumer_thread.join()


Starting consumer...
Produced: {'key': 'message-0', 'value': 'This is message number 0'}
Produced: {'key': 'message-1', 'value': 'This is message number 1'}
Produced: {'key': 'message-2', 'value': 'This is message number 2'}
Consumed: b'{"key": "message-2", "value": "This is message number 2"}'
Produced: {'key': 'message-3', 'value': 'This is message number 3'}
Consumed: b'{"key": "message-3", "value": "This is message number 3"}'
Produced: {'key': 'message-4', 'value': 'This is message number 4'}
Produced: {'key': 'message-5', 'value': 'This is message number 5'}
Produced: {'key': 'message-6', 'value': 'This is message number 6'}
Consumed: b'{"key": "message-6", "value": "This is message number 6"}'
Produced: {'key': 'message-7', 'value': 'This is message number 7'}
Produced: {'key': 'message-8', 'value': 'This is message number 8'}
Produced: {'key': 'message-9', 'value': 'This is message number 9'}
